In [3]:
import sys
import os
import pandas as pd
import numpy as np
from tqdm import trange,tqdm


In [ ]:
#run in SHARK terminal!
compareA1 = pd.read_csv("compareA1_stat.txt",sep='\t',usecols=['SNP','Uniqueness']) #add nrows=10000 if to test
compareA2 = pd.read_csv("compareA2_stat.txt",sep='\t',usecols=['SNP','Uniqueness'])
cleanA1 = compareA1.groupby("SNP",as_index=False).agg(list)
cleanA2 = compareA2.groupby("SNP",as_index=False).agg(list)

def getstr(a):
    a = "".join(set(a))
    a = "".join(set(a))
    return a

cleanA1['strUniqueness'] = cleanA1['Uniqueness'].apply(getstr)
cleanA2['strUniqueness'] = cleanA2['Uniqueness'].apply(getstr)

A1A2 = pd.merge(cleanA1,cleanA2,how='inner',on="SNP",suffixes=['_A1','_A2'])
A1A2['UniquenessPerSNP'] = (A1A2['strUniqueness_A1']+A1A2['strUniqueness_A2']).map(getstr)
A1A2['count_UniquenessPerSNP'] = A1A2['UniquenessPerSNP'].map(len)

# A1A2[A1A2['count_UniquenessPerSNP']>2]
lens = A1A2['count_UniquenessPerSNP']
dict_count = {}
unique_keys = lens.unique()
for i in unique_keys:
    dict_count[i] = (lens==i).sum()
print(dict_count)

#{2: 33002722, 3: 3175338, 4: 4036}

In [4]:
def getloclist(wd,end,filter_out):
    loc_list = []
    EIRA_found = False
    for root, dirs, files in os.walk(wd):
        for file in files:
            if file.endswith(end) and not file.endswith(filter_out):
                if root.split("/")[-2]=="EIRA":
                    loc = os.path.join(root, file)
                    loc_list.insert(0,loc)
                    EIRA_found = True
                else:
                    loc = os.path.join(root, file)
                    loc_list.append(loc) #get bim file location
    if EIRA_found == True:
        print("EIRA detected!")
    else:
        print("Warning! No EIRA bim detected! Use first scanned cohort as ref!")
    print("No of cohorts detected: ",len(loc_list))
    return loc_list
loc_list = getloclist(os.getcwd(),".bim",("_A1corr.bim","_noDup.bim","Prepared.bim"))


EIRA detected!
No of cohorts detected:  14


In [9]:
all_dups = pd.read_table("MultiExclude.txt",header=None)
all_dups

,0
0,10:100000625
1,10:100003242
2,10:100004441
3,10:100004906
4,10:100005282
...,...
3179369,9:99996548
3179370,9:99996634
3179371,9:99997049
3179372,9:99997707


In [5]:
loc_list

['/exports/reum/CKe/Pipelinerun_withPC/EIRA/mergedChr/EIRA_merged.bim',
 '/exports/reum/CKe/Pipelinerun_withPC/RAMS/mergedChr/RAMS_merged.bim',
 '/exports/reum/CKe/Pipelinerun_withPC/Madrid/mergedChr/madrid_merged.bim',
 '/exports/reum/CKe/Pipelinerun_withPC/Manchester/mergedChr/Manchester_merged.bim',
 '/exports/reum/CKe/Pipelinerun_withPC/Tacera/Final_imputed/TACERA_HRCv1_1_FINAL.bim',
 '/exports/reum/CKe/Pipelinerun_withPC/EARTH_B/mergedChr/EARTH_setB_merged.bim',
 '/exports/reum/CKe/Pipelinerun_withPC/Glasgow/mergedChr/Glasgow_merged.bim',
 '/exports/reum/CKe/Pipelinerun_withPC/ACPA/mergedChr/ACPAneg_merged.bim',
 '/exports/reum/CKe/Pipelinerun_withPC/SERA/mergedChr/sera_merged.bim',
 '/exports/reum/CKe/Pipelinerun_withPC/EARTH_A/mergedChr/EA127genotypes_merged.bim',
 '/exports/reum/CKe/Pipelinerun_withPC/PEAC/mergedChr/PEAC_merged.bim',
 '/exports/reum/CKe/Pipelinerun_withPC/Vienna/mergedChr/vienna_merged.bim',
 '/exports/reum/CKe/Pipelinerun_withPC/EAC/mergedChr/EAC_merged.bim']

In [10]:
#generate list of duplicates snps for each cohort based on multiallelics across all cohorts
for run in loc_list:
    bim_cohort = pd.read_csv(run,sep='\t',header=None)
    cohort_name = run.split("/")[-3]
    bim_cohort.rename(columns={0:"CHR",1:"SNP",3:"BP",4:"A1",5:"A2"},inplace=True)   
    if cohort_name != "Tacera":
        bim_cohort['chrbp'] = bim_cohort['SNP'].str.split(":",expand=True)[0]+":"+bim_cohort['SNP'].str.split(":",expand=True)[1]
    else:
        print("process Tacera id for:",cohort_name)
        bim_cohort['chrbp'] = bim_cohort['SNP']
    dup = pd.merge(bim_cohort,all_dups,how="inner",left_on="chrbp",right_on=0) 
    new_loc = run.split(".")[0]+"_dupsExclude.txt"
    dup['SNP'].to_csv(new_loc,sep='\t',index=False,header=False) #should be no duplicates any more
    print("saved dups for: ",os.path.split(new_loc)[1],"excluded snps:",len(dup['SNP']))

KeyboardInterrupt: 

In [6]:
#check how many snps are actually changed after --reference-allele
origin_bim = pd.read_table( '/exports/reum/CKe/Pipelinerun_withPC/EAC/mergedChr/EAC_merged.bim',header=None)

In [13]:
loc_list = getloclist(os.getcwd(),".bim",("_A1corr.bim","_noDup.bim","Prepared.bim"))
loc_list.remove( '/exports/reum/CKe/Pipelinerun_withPC/Glasgow/mergedChr/Glasgow_merged.bim')
loc_list

EIRA detected!
No of cohorts detected:  13


['/exports/reum/CKe/Pipelinerun_withPC/EIRA/mergedChr/EIRA_merged.bim',
 '/exports/reum/CKe/Pipelinerun_withPC/RAMS/mergedChr/RAMS_merged.bim',
 '/exports/reum/CKe/Pipelinerun_withPC/Madrid/mergedChr/madrid_merged.bim',
 '/exports/reum/CKe/Pipelinerun_withPC/Manchester/mergedChr/Manchester_merged.bim',
 '/exports/reum/CKe/Pipelinerun_withPC/Tacera/Final_imputed/TACERA_HRCv1_1_FINAL.bim',
 '/exports/reum/CKe/Pipelinerun_withPC/EARTH_B/mergedChr/EARTH_setB_merged.bim',
 '/exports/reum/CKe/Pipelinerun_withPC/ACPA/mergedChr/ACPAneg_merged.bim',
 '/exports/reum/CKe/Pipelinerun_withPC/SERA/mergedChr/sera_merged.bim',
 '/exports/reum/CKe/Pipelinerun_withPC/EARTH_A/mergedChr/EA127genotypes_merged.bim',
 '/exports/reum/CKe/Pipelinerun_withPC/PEAC/mergedChr/PEAC_merged.bim',
 '/exports/reum/CKe/Pipelinerun_withPC/Vienna/mergedChr/vienna_merged.bim',
 '/exports/reum/CKe/Pipelinerun_withPC/EAC/mergedChr/EAC_merged.bim']

In [11]:
loc_prepared_bim = getloclist(os.getcwd(),"noMAFfilter.assoc.logistic",("noNA_noMAFfilter.assoc.logistic"))
loc_prepared_bim

EIRA detected!
No of cohorts detected:  12


['/exports/reum/CKe/Pipelinerun_withPC/EIRA/mergedChr/gwas_EIRA_flipped_noMAFfilter.assoc.logistic',
 '/exports/reum/CKe/Pipelinerun_withPC/RAMS/mergedChr/gwas_RAMS_flipped_noMAFfilter.assoc.logistic',
 '/exports/reum/CKe/Pipelinerun_withPC/Madrid/mergedChr/gwas_madrid_flipped_noMAFfilter.assoc.logistic',
 '/exports/reum/CKe/Pipelinerun_withPC/Manchester/mergedChr/gwas_Manchester_flipped_noMAFfilter.assoc.logistic',
 '/exports/reum/CKe/Pipelinerun_withPC/Tacera/Final_imputed/gwas_TACERA_noMAFfilter.assoc.logistic',
 '/exports/reum/CKe/Pipelinerun_withPC/EARTH_B/mergedChr/gwas_EARTHB_flipped_noMAFfilter.assoc.logistic',
 '/exports/reum/CKe/Pipelinerun_withPC/ACPA/mergedChr/gwas_ACPA_flipped_noMAFfilter.assoc.logistic',
 '/exports/reum/CKe/Pipelinerun_withPC/SERA/mergedChr/gwas_SERA_flipped_noMAFfilter.assoc.logistic',
 '/exports/reum/CKe/Pipelinerun_withPC/EARTH_A/mergedChr/gwas_EARTHA_flipped_noMAFfilter.assoc.logistic',
 '/exports/reum/CKe/Pipelinerun_withPC/PEAC/mergedChr/gwas_PEAC_f

In [ ]:
for i in range(loc_list):
    origin_bim = pd.read_table(loc_list[i],header=None)
    origin_bim.rename(columns={0:"CHR",1:"SNP",3:"BP",4:"A1",5:"A2"},inplace=True)
    gwas_result = pd.read_table(loc_prepared_bim[i],sep='\t')
    compare_bim = pd.merge(prepared_bim,origin_bim,how='inner',on="SNP",suffixes=('_prepared','_original'))
    printcompare_bim[compare_bim['A1_preapared']==compare_bim['A2_original']][compare_bim['A2_preapared']==compare_bim['A1_original']]

In [12]:
origin_bim.rename(columns={0:"CHR",1:"SNP",3:"BP",4:"A1",5:"A2"},inplace=True)

In [15]:
origin_bim

,CHR,SNP,2,BP,A1,A2
0,1,1:54353:C:A,0,54353,A,C
1,1,1:57183:A:G,0,57183,G,A
2,1,1:62157:G:A,0,62157,A,G
3,1,1:79137:A:T,0,79137,T,A
4,1,1:85108:T:C,0,85108,C,T
...,...,...,...,...,...,...
18801638,22,22:51236700:T:G,0,51236700,G,T
18801639,22,22:51237063:T:C,0,51237063,C,T
18801640,22,22:51237136:G:A,0,51237136,A,G
18801641,22,22:51239586:T:G,0,51239586,G,T


In [15]:
gwas_result = pd.read_table(loc_prepared_bim[0],sep='\t')
# gwas_result.rename(columns={0:"CHR",1:"SNP",3:"BP",4:"A1",5:"A2"},inplace=True)

,0
0,CHR SNP BP A1 TE...
1,1 1:57183:A:G 57183 G A...
2,1 1:62055:G:C 62055 C A...
3,1 1:84139:A:T 84139 T A...
4,1 1:526561:T:G 526561 G A...
...,...
17319199,22 22:51237063:T:C 51237063 C A...
17319200,22 22:51237069:T:C 51237069 C A...
17319201,22 22:51238513:C:G 51238513 G A...
17319202,22 22:51239586:T:G 51239586 G A...


In [16]:
prepared_bim

,CHR,SNP,2,BP,A1,A2
0,1,1:54353:C:A,0,54353,A,C
1,1,1:57183:A:G,0,57183,G,A
2,1,1:62157:G:A,0,62157,A,G
3,1,1:79137:A:T,0,79137,T,A
4,1,1:85108:T:C,0,85108,C,T
...,...,...,...,...,...,...
15622726,22,22:51236700:T:G,0,51236700,G,T
15622727,22,22:51237063:T:C,0,51237063,C,T
15622728,22,22:51237136:G:A,0,51237136,A,G
15622729,22,22:51239586:T:G,0,51239586,G,T


In [17]:
compare_bim = pd.merge(prepared_bim,origin_bim,how='inner',on="SNP",suffixes=('_prepared','_original'))
compare_bim

,CHR_preapared,SNP,2_preapared,BP_preapared,A1_preapared,A2_preapared,CHR_original,2_original,BP_original,A1_original,A2_original
0,1,1:54353:C:A,0,54353,A,C,1,0,54353,A,C
1,1,1:57183:A:G,0,57183,G,A,1,0,57183,G,A
2,1,1:62157:G:A,0,62157,A,G,1,0,62157,A,G
3,1,1:79137:A:T,0,79137,T,A,1,0,79137,T,A
4,1,1:85108:T:C,0,85108,C,T,1,0,85108,C,T
...,...,...,...,...,...,...,...,...,...,...,...
15622726,22,22:51236700:T:G,0,51236700,G,T,22,0,51236700,G,T
15622727,22,22:51237063:T:C,0,51237063,C,T,22,0,51237063,C,T
15622728,22,22:51237136:G:A,0,51237136,A,G,22,0,51237136,A,G
15622729,22,22:51239586:T:G,0,51239586,G,T,22,0,51239586,G,T


In [23]:
compare_bim[compare_bim['A1_preapared']==compare_bim['A2_preapared']]

,CHR_preapared,SNP,2_preapared,BP_preapared,A1_preapared,A2_preapared,CHR_original,2_original,BP_original,A1_original,A2_original


In [21]:
compare_bim[compare_bim['A1_preapared']!=compare_bim['A1_original']]

,CHR_preapared,SNP,2_preapared,BP_preapared,A1_preapared,A2_preapared,CHR_original,2_original,BP_original,A1_original,A2_original
1764,1,1:959231:G:A,0,959231,G,A,1,0,959231,A,G
6847,1,1:1663831:C:T,0,1663831,C,T,1,0,1663831,T,C
6862,1,1:1665613:A:G,0,1665613,A,G,1,0,1665613,G,A
6888,1,1:1671782:C:T,0,1671782,C,T,1,0,1671782,T,C
6919,1,1:1680219:G:A,0,1680219,A,G,1,0,1680219,G,A
...,...,...,...,...,...,...,...,...,...,...,...
15612977,22,22:49798819:A:G,0,49798819,G,A,22,0,49798819,A,G
15618946,22,22:50643629:G:A,0,50643629,G,A,22,0,50643629,A,G
15619472,22,22:50720622:C:T,0,50720622,C,T,22,0,50720622,T,C
15620493,22,22:50881518:A:G,0,50881518,G,A,22,0,50881518,A,G


In [25]:
# a1s are flipped properly
compare_bim[compare_bim['A1_preapared']==compare_bim['A2_original']][compare_bim['A2_preapared']==compare_bim['A1_original']]

/tmp/ipykernel_1831982/1066419545.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  compare_bim[compare_bim['A1_preapared']==compare_bim['A2_original']][compare_bim['A2_preapared']==compare_bim['A1_original']]


,CHR_preapared,SNP,2_preapared,BP_preapared,A1_preapared,A2_preapared,CHR_original,2_original,BP_original,A1_original,A2_original
1764,1,1:959231:G:A,0,959231,G,A,1,0,959231,A,G
6847,1,1:1663831:C:T,0,1663831,C,T,1,0,1663831,T,C
6862,1,1:1665613:A:G,0,1665613,A,G,1,0,1665613,G,A
6888,1,1:1671782:C:T,0,1671782,C,T,1,0,1671782,T,C
6919,1,1:1680219:G:A,0,1680219,A,G,1,0,1680219,G,A
...,...,...,...,...,...,...,...,...,...,...,...
15612977,22,22:49798819:A:G,0,49798819,G,A,22,0,49798819,A,G
15618946,22,22:50643629:G:A,0,50643629,G,A,22,0,50643629,A,G
15619472,22,22:50720622:C:T,0,50720622,C,T,22,0,50720622,T,C
15620493,22,22:50881518:A:G,0,50881518,G,A,22,0,50881518,A,G
